<a href="https://colab.research.google.com/github/contactaman5/NLP-with-ML/blob/main/sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install --quiet nltk[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00


In [29]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [22]:
data_dir = '/content/drive/MyDrive/Data Science/NLP/data/cleaned data.csv'
data = pd.read_csv(data_dir)

In [23]:
data.head(10)

,Text,Classification
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,joy
9,i feel romantic too,love


In [24]:
data['Classification'].value_counts()

,count
Classification,
joy,5822
sadness,4866
anger,2244
fear,2076
love,1350
surprise,605


## Data Processing

In [25]:
sentences = data['Text'].values.tolist()
labels = data['Classification'].values.tolist()

In [26]:
sentences[:10]

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy',
 'ive been feeling a little burdened lately wasnt sure why that was',
 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
 'i feel as confused about life as a teenager or as jaded as a year old man',
 'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
 'i feel romantic too']

In [10]:
# converting to lower case
sentences = [token.lower() for token in sentences]

In [31]:
# tokenize
sentences = [sentence.split(' ') for sentence in sentences]

In [34]:
# removing stopwords
eng_stopwords = stopwords.words('english')

sentences = [[token for token in sentence if token not in eng_stopwords] for sentence in sentences]

In [37]:
sentences[5]

['ive', 'feeling', 'little', 'burdened', 'lately', 'wasnt', 'sure']

In [38]:
# Stemming

from nltk.stem import PorterStemmer

ps = PorterStemmer()

sentences = [[ps.stem(token) for token in sentence] for sentence in sentences]

In [41]:
# joining tokens
sentences = [' '.join(sentence) for sentence in sentences]

In [44]:
sentences[1]

'go feel hopeless damn hope around someon care awak'

## Creating a Vocabulary

In [48]:
# Creating a vocabulary dictionary
from collections import defaultdict

vocab = defaultdict(int)

for sentence in sentences:
  words = re.findall(r'\b\w+\b', sentence)
  for word in words:
    vocab [word] += 1

sorted_vocab = dict(sorted(vocab.items(), key=lambda x: x[1], reverse=True))

print(sorted_vocab)

{'feel': 17959, 'like': 3148, 'im': 2552, 'get': 1043, 'time': 1019, 'know': 986, 'make': 983, 'realli': 972, 'go': 934, 'want': 896, 'love': 827, 'littl': 786, 'think': 758, 'peopl': 712, 'day': 711, 'one': 690, 'thing': 673, 'would': 669, 'even': 631, 'ive': 623, 'still': 622, 'life': 567, 'way': 560, 'bit': 556, 'need': 542, 'someth': 531, 'much': 520, 'work': 501, 'dont': 496, 'could': 484, 'start': 476, 'say': 459, 'tri': 433, 'look': 428, 'back': 426, 'see': 422, 'good': 422, 'pretti': 415, 'come': 380, 'right': 378, 'alway': 376, 'help': 360, 'today': 360, 'also': 358, 'year': 356, 'friend': 349, 'take': 342, 'around': 323, 'use': 322, 'person': 317, 'cant': 314, 'well': 303, 'made': 303, 'happi': 299, 'though': 295, 'live': 293, 'hate': 290, 'got': 289, 'didnt': 285, 'felt': 283, 'someon': 277, 'never': 274, 'thought': 274, 'write': 273, 'hope': 272, 'lot': 271, 'week': 270, 'quit': 264, 'find': 262, 'mani': 262, 'read': 261, 'everi': 259, 'sure': 258, 'less': 255, 'enough': 24

## Vectorization using Bag of Words

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

# Text Vectorization
cv = CountVectorizer()

X_cv = cv.fit_transform(sentences)

In [56]:
print("Vocabulary:", cv.get_feature_names_out())

Vocabulary: ['aa' 'aaaaaaand' 'aaaaand' ... 'zum' 'zumba' 'zz']


## Vectorization using TFIDF

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_tfidf = tfidf.fit_transform(sentences)

# Modelling using Count Vectorizer

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

X = X_cv.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, labels,
                                                    test_size = 0.2,
                                                    random_state=32)

print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

(13570, 10476) 13570
(3393, 10476) 3393
              precision    recall  f1-score   support

       anger       0.87      0.65      0.75       465
        fear       0.83      0.63      0.72       416
         joy       0.72      0.92      0.81      1133
        love       0.76      0.28      0.41       268
     sadness       0.76      0.92      0.83       973
    surprise       0.72      0.09      0.17       138

    accuracy                           0.76      3393
   macro avg       0.78      0.58      0.61      3393
weighted avg       0.77      0.76      0.74      3393



In [76]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train_num = le.fit_transform(y_train)
y_test_num = le.transform(y_test)

In [85]:
# print("Classes: ", le.classes_)
# print("Encoded Vals: ", le.transform(le.classes_))

print('Target Encoding')
target_dict = dict(zip(le.classes_, le.transform(le.classes_)))
target_dict

Target Encoding


{'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}

In [87]:
# XG Boost

from xgboost import XGBClassifier

model = XGBClassifier()

model.fit(X_train, y_train_num)

y_pred = model.predict(X_test)

print(classification_report(y_test_num, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       465
           1       0.82      0.85      0.83       416
           2       0.88      0.91      0.89      1133
           3       0.73      0.71      0.72       268
           4       0.93      0.87      0.90       973
           5       0.69      0.70      0.70       138

    accuracy                           0.86      3393
   macro avg       0.82      0.82      0.82      3393
weighted avg       0.86      0.86      0.86      3393



## Modelling using TFIDF Vectorizer

In [89]:

X = X_tfidf.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, labels,
                                                    test_size = 0.2,
                                                    random_state=32)

print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

(13570, 10476) 13570
(3393, 10476) 3393
              precision    recall  f1-score   support

       anger       0.97      0.31      0.47       465
        fear       0.89      0.27      0.41       416
         joy       0.59      0.98      0.74      1133
        love       1.00      0.04      0.07       268
     sadness       0.71      0.90      0.80       973
    surprise       0.00      0.00      0.00       138

    accuracy                           0.66      3393
   macro avg       0.69      0.42      0.41      3393
weighted avg       0.72      0.66      0.60      3393



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [90]:
# XG Boost

from xgboost import XGBClassifier

model = XGBClassifier()

model.fit(X_train, y_train_num)

y_pred = model.predict(X_test)

print(classification_report(y_test_num, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87       465
           1       0.83      0.82      0.83       416
           2       0.83      0.91      0.87      1133
           3       0.73      0.65      0.69       268
           4       0.92      0.86      0.89       973
           5       0.71      0.72      0.72       138

    accuracy                           0.85      3393
   macro avg       0.82      0.80      0.81      3393
weighted avg       0.85      0.85      0.85      3393

